In [2]:
import pandas as pd
import numpy as np
import os
import glob
from collections import Counter
import math
from bitarray import bitarray
import mmh3
import matplotlib.pyplot as plt
from __future__ import division
plt.style.use('ggplot')
%matplotlib inline

In [3]:
df_ip = pd.read_table("C:\Users\Apourva\Downloads\ips.txt", header = None)
iplist = df_ip[0].tolist()

In [4]:
l1 = int(math.floor((len(iplist))/2))
l2 = int(math.ceil((len(iplist))/2))
training = iplist[:l1]
test = iplist[l2:]

print len(training) + len(test)
print len(training)

3414010
1707005


In [5]:
l = [1,2,3,2,4]
print set(l)

set([1, 2, 3, 4])


In [5]:
match_test_ip_in_training = {}
intr = list(set(training).intersection(set(test)))
print 'Number of IP addresses common in Test and Training: ', len(intr)

Number of IP addresses common in Test and Training:  50273


In [7]:
def bloom_filter ( n ):
    bit_array = bitarray(n)
    bit_array.setall(0)
    for i in training:
        b1 = mmh3.hash(str(i), 42) % n 
        bit_array[b1] = 1
        b2 = mmh3.hash(str(i), 24) % n 
        bit_array[b2] = 1
        b3 = mmh3.hash(str(i), 84) % n 
        bit_array[b3] = 1  
    return bit_array

In [8]:
def calculate_false_positives ( bit_array, n ):
    found_list = []
    false_positive_count = 0
    for i in test:
        b1 = mmh3.hash(i, 42) % n 
        b2 = mmh3.hash(i, 24) % n 
        b3 = mmh3.hash(i, 84) % n
        x = bit_array[b1] == 1 and bit_array[b2] == 1 and bit_array[b3] == 1
        if x:
            found_list.append(i)
    false_positive_list = list(set(found_list) - set(intr))
    return false_positive_list

In [42]:
def false_positive_rate_theoretical_bound ( m, n ):
    t0 = (m * np.log(2)) / n
    t1 = - t0 * (n / m)
    t2 = np.exp(t1)
    t3 = 1 - t2
    t4 = np.power(t3, t0)
    print t4

In [43]:
n = 10000
bit_array_10k = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_10k, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  200825
0.986794595327


In [44]:
n = 100000
bit_array_100k = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_100k, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  200815
false positive rate:  0.875523125894
None


In [45]:
n = 1000000
bit_array_1M = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_1M, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  57833
false positive rate:  0.26465263318
None


In [46]:
n = 10000000
bit_array_10M = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_10M, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  209
false positive rate:  1.6856297745e-06
None


In [13]:
n = 15000000
bit_array_15M = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_15M, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  75


In [47]:
n = 1000000000
bit_array_1B = bloom_filter(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_1B, n))
print "false positive rate: ", false_positive_rate_theoretical_bound(n, 361423)

number of false positives:  0
false positive rate:  0.0
None


We split the list of IP addresses into two equal halves. Let's call the first half as training dataset and the second hafl as test dataset. We create a Bloom Filter for hash sizes of 10K, 100K, 1M, 10M and 1B using three seed values. The IP addresses in the training dataset are mapped to the hash spaces of different sizes.

For each IP address in the test dataset, we check if it's present in the training dataset by applying the hash functions on the IP addresses in the test dataset.

In this dataset, there are actually 50273 IP addresses in the Test dataset which are also present in the training dataset. Now we check how many IP addresses in the Test dataset has been found in the training data by the Bloom Filter. If the Bloom Filter has found an IP address in the hash space though it doesn't actualy exist, then this is a flase positive.

The table below shows the number of false positives generated for different sizes of hash space. 

The probability for false positives increases when the size of the hash space is much smaller that the number of distinct values to be mapped to the space. In this case, an IP address in the test data that is not actually present in the training data may get hashed to the same value as an IP address that is indeed present leading to a false positive.

However, as the size of the hash space increases, we see that the number of coolisions is lower and for a hash size of 1 Billion, there are no false positives.

| Hash Space Size | Number of False Positives |False Positive Rate |
|-----------------|---------------------------|--------------------|
| 10000           | 200825                    |  0.986794595327    |
| 100000          | 200815                    |  0.875523125894    |
| 1000000         | 57833                     |  0.26465263318     |
| 10000000        | 209                       |  1.6856297745e-06  |
| 1000000000      | 0                         |  0.0               |

The more hash functions you have, the slower your bloom filter, and the quicker it fills up. If you have too few, however, you may suffer too many false positives.
To optimize the number of hash functions 'k' and size on bit array 'm' based on the number of distint IP addresses to be hashed 'n' using the formulas below:


 m = - n * (ln p) / (ln 2)^2

 k = - (ln p) / (ln 2)

In [13]:
l2 = np.log(2) * np.log(2)
# We choose p = 0.01
lp = np.log(0.01)

In [41]:
# Get number of distinct IP addresses in training data
n = len(set(training))
print n
# Size of bit array
m = - ( n * lp ) / l2
print "bit array size: ", m

# No of hash functions to use 
k = - np.log(0.001) / np.log(2)
print "number of hash functions: ", k

361423
bit array size:  3464260.55392
number of hash functions:  9.96578428466


In [39]:
def bloom_filter_10 ( n ):
    bit_array = bitarray(n)
    bit_array.setall(0)
    for i in training:
        b1 = mmh3.hash(str(i), 42) % n 
        bit_array[b1] = 1
        b2 = mmh3.hash(str(i), 27) % n 
        bit_array[b2] = 1
        b3 = mmh3.hash(str(i), 89) % n 
        bit_array[b3] = 1  
        b4 = mmh3.hash(str(i), 30) % n 
        bit_array[b4] = 1  
        b5 = mmh3.hash(str(i), 16) % n 
        bit_array[b5] = 1  
        b6 = mmh3.hash(str(i), 68) % n 
        bit_array[b6] = 1  
        b7 = mmh3.hash(str(i), 75) % n 
        bit_array[b7] = 1  
        b8 = mmh3.hash(str(i), 19) % n 
        bit_array[b8] = 1  
        b9 = mmh3.hash(str(i), 8) % n 
        bit_array[b9] = 1  
        b10 = mmh3.hash(str(i), 53) % n 
        bit_array[b10] = 1  
    return bit_array

In [38]:
def calculate_false_positives_10 ( bit_array, n ):
    found_list = []
    false_positive_count = 0
    for i in test:
        b1 = mmh3.hash(i, 42) % n 
        b2 = mmh3.hash(i, 24) % n 
        b3 = mmh3.hash(i, 84) % n
        x = bit_array[b1] == 1 and bit_array[b2] == 1 and bit_array[b3] == 1 and bit_array[b4] and bit_array[b5] and bit_array[b6] and bit_array[b7] and  bit_array[b8] and bit_array[b9] and bit_array[b10] 
        if x:
            found_list.append(i)
    false_positive_list = list(set(found_list) - set(intr))
    return false_positive_list

In [40]:
n = 3464261
bit_array_optimal = bloom_filter_10(n)
print "number of false positives: ", len(calculate_false_positives (bit_array_optimal, n))

number of false positives:  54611


For number of distinct elements expected in the stream n = 361423 we calculate the optimal size of bit array for error rate of 0.001 as  m = 3464261 and the number of hash functions as 10 (rounded from 9.96). We see that the number of false positives obtained is 54611

In [49]:
n = len(set(training)) + len(set(test))
n

612522

In [57]:
w = 1000000
d = 6
mat = np.zeros([d, w])
seeds = np.random.randint(1, 100, d, dtype='l')

In [58]:
def CM_Sketch ( w, d ):  
    for ip in iplist:
        for i in range(d):
            b = mmh3.hash(str(ip),seeds[i]) % w
            mat[i][b] += 1

In [60]:
CM_Sketch(w, d)
print mat.shape

(6L, 1000000L)


In [61]:
def get_count (ip, w, d):
    counts = np.zeros(d)
    for i in range(d):
        b = mmh3.hash(ip, seeds[i]) % w 
        counts[i] = mat[i][b]
        # print "b, freq", b, mat[i][b]
    return np.ndarray.min(counts)

In [62]:
ip_freq = {}
for ip in set(iplist):
    ip_freq[ip] = get_count(ip, w, d)

In [49]:
t = [('115.177.11.215', 294690),
 ('115.176.182.196', 259241),
 ('192.3.106.42', 66572),
 ('191.96.249.189', 51251),
 ('104.192.0.20', 18707),
 ('198.204.234.26', 6817),
 ('198.204.234.27', 5284),
 ('198.204.234.28', 5239),
 ('185.93.185.10', 5157),
 ('198.204.234.30', 5049)]

In [63]:
for row in range(10):
    print t[row][0], t[row][1], ip_freq[t[row][0]]

115.177.11.215 294690 549182.0
115.176.182.196 259241 259241.0
192.3.106.42 66572 89719.0
191.96.249.189 51251 51251.0
104.192.0.20 18707 37414.0
198.204.234.26 6817 12887.0
198.204.234.27 5284 9793.0
198.204.234.28 5239 9677.0
185.93.185.10 5157 5157.0
198.204.234.30 5049 9311.0
